In [1]:
from pprint import pprint
from pathlib import Path
import pandas as pd

In [2]:
MARKET = "csi300"
BENCHMARK = "SH000300"
EXP_NAME = "/home/zwc/tutorial_exp"

In [3]:
from qlib.tests.data import GetData
GetData().qlib_data(exists_skip=True)

2022-10-20 23:29:21.594 | WARNING  | qlib.tests.data:qlib_data:150 - Data already exists: ~/.qlib/qlib_data/cn_data, the data download will be skipped
	If downloading is required: `exists_skip=False` or `change target_dir`


In [4]:
import qlib
qlib.init()

[41537:MainThread](2022-10-20 23:29:21,604) INFO - qlib.Initialization - [config.py:402] - default_conf: client.
[41537:MainThread](2022-10-20 23:29:21,888) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[41537:MainThread](2022-10-20 23:29:21,889) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/Users/linweiqiang/.qlib/qlib_data/cn_data')}


In [5]:
import tradercompany.mytools as tools
#tools.mytools.make_peak(filename="test2.csv")


In [6]:
import pandas as pd

In [7]:
df=tools.mytools.read_pkl(filename="test.pkl")

In [8]:
import os, sys
import pandas as pd
import numpy as np
import datetime
from glob import glob
from tradercompany import Company
from tradercompany.aggregations import top_average,simple_average
from tradercompany.transformer import StockDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

#import argparse
import math


In [9]:
model = Company(
    n_traders=1,
    n_features=20, #表示列的数量这里为15个
    batch_size=10,
    max_terms=15,
    max_lag=9,#最大滞后，表示每次卷积前面9个的数据，就是用-10倒0的数据来计算下一天的结果
    educate_pct=50,#教育的参数，表示后面百分50的人要接受教育
    eval_lookback=2,#倒多少个batch_size用于计算分数#用后面验证的数据来计算socre
    eval_method='acc',
    aggregate=simple_average#聚合函数用什么
)

[41537:MainThread](2022-10-20 23:29:25,793) INFO - qlib.TransformerModel - [Model.py:64] - Naive Transformer:
batch_size : 1
device : cpu


In [10]:
df_train=df[0]
df_valid=df[1]

In [11]:
pred = model.dynamic_predict(df_train, df_valid,total_epoch=1,save_path="sss.pkl")

[41537:MainThread](2022-10-20 23:29:26,915) INFO - qlib.TransformerModel - [myTrader.py:89] - start...
[41537:MainThread](2022-10-20 23:29:26,915) INFO - qlib.TransformerModel - [Company.py:118] - Epoch0:
[41537:MainThread](2022-10-20 23:29:26,917) INFO - qlib.TransformerModel - [myTrader.py:201] - start...
[41537:MainThread](2022-10-20 23:29:26,918) INFO - qlib.TransformerModel - [myTrader.py:205] - training...
[41537:MainThread](2022-10-20 23:29:58,527) INFO - qlib.TransformerModel - [myTrader.py:207] - evaluating...
[41537:MainThread](2022-10-20 23:30:01,208) INFO - qlib.TransformerModel - [myTrader.py:210] - train -0.380774, valid -0.347803
[41537:MainThread](2022-10-20 23:30:01,222) INFO - qlib.TransformerModel - [myTrader.py:82] - best score: -0.347803


In [12]:
model.traders[0].Model=tools.mytools.read_pkl("sss.pkl")

In [13]:
x_train_values=df_valid['feature'].values

In [14]:
indices = np.arange(len(x_train_values))

In [15]:
import torch

In [16]:
feature = torch.from_numpy(df_valid['feature'].values[indices[0 :1]]).float()

In [20]:
model.traders[0].Model.model(feature)

tensor(-0.0302, grad_fn=<SqueezeBackward0>)

In [ ]:
###################################
# prediction, backtest & analysis
###################################
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs": {
            "signal": "<PRED>",
            "topk": 50,
            "n_drop": 5,
        },
    },
    "backtest": {
        "start_time": "2017-01-01",
        "end_time": "2020-08-01",
        "account": 100000000,
        "benchmark": BENCHMARK,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
with R.start(experiment_name=EXP_NAME, recorder_id=rid, resume=True):

    # signal-based analysis
    rec = R.get_recorder()
    sar = SigAnaRecord(rec)
    sar.generate()

    #  portfolio-based analysis: backtest
    par = PortAnaRecord(rec, port_analysis_config, "day")
    par.generate()